In [2]:
import docker
import os
import requests
import sys

from dotenv import load_dotenv

In [3]:
sys.version

'3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]'

In [4]:
docker_model_endpoint = "http://127.0.0.1:5000"

In [5]:
load_dotenv("azure.env")

key = os.getenv("key")

In [6]:
try:
    response = requests.get(docker_model_endpoint)
    respcode = response.status_code
    if respcode == 200:
        print(f"OK. Status code = {respcode}")
    else:
        print(f"Failed to get status. Status code: {respcode}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

OK. Status code = 200


In [7]:
docker_client = docker.from_env()
docker_client

In [8]:
print("Active Docker container(s):\n")

for idx2, container in enumerate(docker_client.containers.list(), start=1):
    print(f"{idx2} - Docker container name: {container.name}")
    print(f"Container image: {container.image}")
    print(f"Container id: {container.id}\n")

Active Docker container(s):

1 - Docker container name: peaceful_hermann
Container image: <Image: 'mcr.microsoft.com/azure-cognitive-services/translator/text-translation:latest'>
Container id: 11d4b6c128f4e0898b00b7e3fd340fdc1d88ba383fbb0e7327115ca77b516f45



In [9]:
def get_text_translation_from_docker(from_lang, to_lang, text):
    """
    Translates text from one language to another using a local Dockerized 
    translation service.

    Args:
        from_lang (str): The language code of the source text (e.g., 'en' for English).
        to_lang (str): The language code of the target text (e.g., 'es' for Spanish).
        text (str): The text to be translated.

    Returns:
        None

    This function sends a POST request to the local translation service running on Docker,
    retrieves the translated text, and prints it in a formatted JSON structure.
    """
    docker_translator_url = f'{docker_model_endpoint}/translate?api-version=3.0&from={from_lang}&to={to_lang}'
    headers = {'Content-Type': 'application/json'}
    body = [{'text': text}]

    # Send the request to Docker
    request = requests.post(docker_translator_url, headers=headers, json=body)
    # Get the result
    response = request.json()

    return response[0]["translations"][0]["text"]

## Tests

In [10]:
text = """Microsoft Azure uses large-scale virtualization at Microsoft data centers worldwide and offers more \
than 600 services.[11] Microsoft Azure offers a service level agreement (SLA) that guarantees 99.9% \
availability for applications and data hosted on its platform, subject to specific terms and conditions \
outlined in the SLA documentation."""

print(f"Text to translate:\n\n{text}")

Text to translate:

Microsoft Azure uses large-scale virtualization at Microsoft data centers worldwide and offers more than 600 services.[11] Microsoft Azure offers a service level agreement (SLA) that guarantees 99.9% availability for applications and data hosted on its platform, subject to specific terms and conditions outlined in the SLA documentation.


In [11]:
get_text_translation_from_docker("en", "fr", text)

'Microsoft Azure utilise la virtualisation à grande échelle dans les centres de données Microsoft du monde entier et offre plus de 600 services. [11] Microsoft Azure propose un contrat de niveau de service (SLA) qui garantit une disponibilité de 99,9 % pour les applications et les données hébergées sur sa plateforme, sous réserve de conditions générales spécifiques décrites dans la documentation SLA.'

In [12]:
get_text_translation_from_docker("en", "es", text)

'Microsoft Azure utiliza la virtualización a gran escala en los centros de datos de Microsoft en todo el mundo y ofrece más de 600 servicios. [11] Microsoft Azure ofrece un acuerdo de nivel de servicio (SLA) que garantiza una disponibilidad del 99,9 % para las aplicaciones y los datos alojados en su plataforma, sujeto a los términos y condiciones específicos descritos en la documentación del SLA.'

In [13]:
text = """Expande tus habilidades de IA con las tecnologías más recientes presentadas en Microsoft Ignite.\
Paga por uso o prueba Azure gratis hasta 30 días. No hay ningún compromiso por adelantado; puedes cancelar \
en cualquier momento."""

print(f"Text to translate:\n\n{text}")

Text to translate:

Expande tus habilidades de IA con las tecnologías más recientes presentadas en Microsoft Ignite.Paga por uso o prueba Azure gratis hasta 30 días. No hay ningún compromiso por adelantado; puedes cancelar en cualquier momento.


In [14]:
get_text_translation_from_docker("es", "fr", text)

'Développez vos compétences en IA avec les dernières technologies présentées dans Microsoft Ignite.Payez au fur et à mesure ou essayez Azure gratuitement pendant 30 jours maximum. Il n’y a pas d’engagement initial\xa0; Vous pouvez annuler à tout moment.'

In [15]:
get_text_translation_from_docker("es", "en", text)

'Expand your AI skills with the latest technologies introduced at Microsoft Ignite.Pay as you go or try Azure free for up to 30 days. There is no upfront commitment; You can cancel at any time.'